# Expérimentation projet ADS #

### Optimisation algorithmique de trajets de drones de livraisons ###

Langage : Python 3.11.1

Algorithmes : 
- Génétique
- Artifical Bee Colony (ABC)
- Particle Swarm Optimization (PSO)

### Schéma et graphe du contexte ###

<div style="display:flex; justify-content:space-evenly; align-items:center; padding:10px;">
    <div style="display:flex; flex-direction:column; width:350px">
        <img src="Schéma/schema_contexte.png" alt="Schéma" style="flex:1; background-color:white; width="250px">
        <figcaption style="flex:1; background-color:white; text-align:center; color:black;">Schéma du contexte</figcaption>
    </div>
    <div style="display:flex; flex-direction:column; width:350px">
        <img src="Schéma/graphe_contexte.png" alt="Graphe" style="flex:1; background-color:white; width="250px">
        <figcaption style="flex:1; background-color:white; text-align:center; color:black;">Graphe du contexte</figcaption>
    </div>
</div>

In [42]:
# Imports
import numpy as np
import time
import random as rand
import matplotlib.pyplot as plt

In [43]:
# Contexte de l'étude du graphe ("/Schéma/graphe_contexte.png") sous forme du code pour résolution du problème du VRPPD

# Paramètres d'entrés
nb_entrepots = 1
nb_clients = 8

nb_drone = 2
capacite_drone = 1
vitesse_drone = 60

# Matrice d'adjacence du graphe
matrice_distance = np.array([
    [0, 5.4, 3.2, 10.1, 7.6, 2.1, 14.4, 9.8, 4.6],
    [5.4, 0, 0, 0, 0, 0, 0, 0, 0],
    [3.2, 0, 0, 0, 0, 0, 0, 0, 0],
    [10.1, 0, 0, 0, 0, 0, 0, 0, 0],
    [7.6, 0, 0, 0, 0, 0, 0, 0, 0],
    [2.1, 0, 0, 0, 0, 0, 0, 0, 0],
    [14.4, 0, 0, 0, 0, 0, 0, 0, 0],
    [9.8, 0, 0, 0, 0, 0, 0, 0, 0],
    [4.6, 0, 0, 0, 0, 0, 0, 0, 0]
])

# Temps d'execution maximum
max_time = 15

In [44]:
# Fonctions communes à tous les algorithmes

def generate_random_solution(same_size=False):
    # Compute the total distance
    total_distance = matrice_distance.sum()
    
    # Compute the target distance per vehicle
    target_distance_per_vehicle = total_distance / nb_drone
    
    # Initialize the groups
    groups = [[] for i in range(nb_drone)]
    group_distances = [0 for i in range(nb_drone)]
    
    # Assign the customers to the groups
    for i in range(1, matrice_distance.shape[0]):
        # Compute the customer distances to each group
        distances = [abs(matrice_distance[i,j]) for j in range(nb_drone)]
        
        # Compute the weights for each group based on the distance already covered
        weights = [1.0 - (group_distances[j] / target_distance_per_vehicle) for j in range(nb_drone)]
        
        # Normalize the weights so they sum to 1
        total_weight = sum(weights)
        weights = [w / total_weight for w in weights]
        
        # Choose the group with the highest weight
        group_idx = np.random.choice(range(nb_drone), p=weights)
        
        # Add the customer to the group
        groups[group_idx].append(i)
        group_distances[group_idx] += distances[group_idx]

    # Add 0 at the end, the beginning and between each group
    for i in range(len(groups)):
        for j in range(len(groups[i])-1):
            groups[i].insert(2*j+1,0)
            j += 1
        groups[i].insert(0,0)
        groups[i].append(0)

    if same_size:
        # Check if the subgroups are the same size
        if len(groups[0]) != len(groups[1]):
            # Add as many 0 as necessary to the smallest group
            if len(groups[0]) > len(groups[1]):
                for i in range(len(groups[0])-len(groups[1])):
                    groups[1].insert(-1,0)
            else:
                for i in range(len(groups[1])-len(groups[0])):
                    groups[0].insert(-1,0)
        
    return groups

def calculate_fitness_vrppd(solution):
    # Calculate the total distance for each vehicle
    total_distances = []
    for vehicle in range(nb_drone):
        total_distance = 0
        for i in range(len(solution[vehicle])):
            total_distance += matrice_distance[solution[vehicle][i-1], solution[vehicle][i%len(solution[vehicle])]]
        total_distances.append(round(total_distance,1))

    # Calculate the mean of the total distances
    mean_distance = sum(total_distances) / nb_drone

    # Calculate the fitness score based on the difference between the smallest and largest distance
    min_distance = min(total_distances)
    max_distance = max(total_distances)
    fitness_score = 1 - abs(max_distance - min_distance) / (mean_distance / 2)

    return total_distances, fitness_score, solution

def print_final_result(solution, fitness, total_distance, iteration_times):
    print("Solution:", solution)
    print("Fitness:", round(fitness, 6))

    print("\nDistance totale :", sum(total_distance))
    for i in range(nb_drone):
        print("Distance drone", i+1, ":", round(total_distance[i],1))

    print("\nTemps d'exécution :", round(sum(iteration_times)), "secondes")
    print("Nombre d'itérations :", len(iteration_times))

    print("\nTemps moyen par itération :", round(sum(iteration_times) / len(iteration_times), 6), "secondes")
    print("Itération par seconde :", round(len(iteration_times) / sum(iteration_times)), "itérations par seconde")
    print("Itération la plus rapide :", round(min([x for x in iteration_times if x != 0.0]), 6), "secondes")
    print("Itération la plus longue :", round(max(iteration_times), 6), "secondes")


solution_test = generate_random_solution()
total_distance_test, fitness_score_test, solution_calc_test = calculate_fitness_vrppd(solution_test)
print("Solution :", solution_calc_test)
print("Distance totale :", total_distance_test, "soit", round(sum(total_distance_test),1))
print("Score de fitness :", fitness_score_test)

Solution : [[0, 1, 0, 2, 0, 3, 0, 6, 0, 7, 0], [0, 4, 0, 5, 0, 8, 0]]
Distance totale : [85.8, 28.6] soit 114.4
Score de fitness : -0.9999999999999998


In [45]:
# Bruteforce

def bruteforce():
    # Initialize the best solution
    best_solution = None
    best_fitness_score = 0
    best_total_distance = 0
    
    # Initialize the timer
    start_time = time.time()
    iteration_times = []
    
    # Iterate over all possible solutions
    for i in range(9999999999):
        current_iteration_time = time.time()
        # Generate a new solution
        solution = generate_random_solution()
        
        # Calculate the fitness score for the solution
        total_distance, fitness_score, solution_calc = calculate_fitness_vrppd(solution)
        
        # Check if the solution is better than the current best
        if fitness_score > best_fitness_score:
            best_solution = solution_calc
            best_fitness_score = fitness_score
            best_total_distance = total_distance

        iteration_times.append(time.time() - current_iteration_time)
        
        # Check if the time limit has been reached
        if time.time() - start_time > max_time:
            break
    
    return best_solution, best_fitness_score, best_total_distance, iteration_times

best_solution_bruteforce, best_fitness_score_bruteforce, total_distance_bruteforce, iteration_times = bruteforce()
print_final_result(best_solution_bruteforce, best_fitness_score_bruteforce, total_distance_bruteforce, iteration_times)

Solution: [[0, 1, 0, 2, 0, 3, 0, 7, 0], [0, 4, 0, 5, 0, 6, 0, 8, 0]]
Fitness: 0.986014

Distance totale : 114.4
Distance drone 1 : 57.0
Distance drone 2 : 57.4

Temps d'exécution : 15 secondes
Nombre d'itérations : 102598

Temps moyen par itération : 0.000146 secondes
Itération par seconde : 6863 itérations par seconde
Itération la plus rapide : 5.2e-05 secondes
Itération la plus longue : 0.024324 secondes


In [46]:
# Algorithme génétique

def mutation(solution):
    # Determine the length of the shortest list in the solution and the amount of swaps to perform
    min_len = min(len(solution[0]), len(solution[1]))
    num_swaps = rand.randint(1, min_len)
    
    # Choose random indices in the lists to swap
    indices = rand.sample(range(0, min_len), num_swaps)
    
    # Swap the values at the chosen indices
    for i in indices:
        # Choose a random index to swap with
        j = rand.choice([x for x in range(min_len) if x != i])
        solution[0][i], solution[1][j] = solution[1][j], solution[0][i]
    
    return solution

def genetic_algorithm_vrppd(population_size, elite_size):    
    # Initialize the population with random solutions
    population = [generate_random_solution() for i in range(population_size)]
        
    # Keep track of the best solution and its fitness
    best_total_distance, best_fitness, best_solution = calculate_fitness_vrppd(population[0])
    
    # Initialize the timer
    start_time = time.time()
    iteration_times = []

    # Evolution loop
    while True:
        current_iteration_time = time.time()
        
        # Evaluate the fitness of each solution
        fitness_scores = []
        for solution in population:
            total_distance, fitness, returned_sol = calculate_fitness_vrppd(solution)
            fitness_scores.append((returned_sol, fitness))
        
            # Update the best solution if necessary
            if fitness > best_fitness:
                best_fitness = fitness
                best_solution = returned_sol
                best_total_distance = total_distance

        # Sort solutions by fitness and keep the elite solutions
        fitness_scores.sort(key=lambda x: x[1])
        elite = [x[0] for x in fitness_scores[:elite_size]]

        # Create a new population by performing mutation
        new_population = elite.copy()
        while len(new_population) < population_size:
            
            # Select a random parent
            parent = rand.choice(elite)
            
            # Create a mutant child
            child = mutation(parent)
            
            # Add the child to the new population
            new_population.append(child)

        # Replace the old population with the new population
        population = new_population

        # Add the iteration time to the list
        iteration_times.append(time.time() - current_iteration_time)

        # Check if the evolution loop should stop
        if time.time() - start_time > max_time:
            break
        
    return best_solution, best_fitness, best_total_distance, iteration_times

# Résultat
best_solution, best_fitness, best_total_distance, iteration_times = genetic_algorithm_vrppd(population_size=50, elite_size=5)
print_final_result(best_solution, best_fitness, best_total_distance, iteration_times)

Solution: [[0, 6, 0, 7, 0, 8, 0], [0, 1, 0, 2, 0, 3, 0, 4, 0, 5, 0]]
Fitness: 0.972028

Distance totale : 114.4
Distance drone 1 : 57.6
Distance drone 2 : 56.8

Temps d'exécution : 15 secondes
Nombre d'itérations : 18164

Temps moyen par itération : 0.000825 secondes
Itération par seconde : 1212 itérations par seconde
Itération la plus rapide : 0.000118 secondes
Itération la plus longue : 0.01563 secondes


In [47]:
# Artificial Bee Colony

def swap(solution):
    # Choose one random odd index in each list
    i = rand.choice([x for x in range(len(solution[0])) if x % 2 == 1])
    j = rand.choice([x for x in range(len(solution[1])) if x % 2 == 1])

    # Swap the values at the chosen indices
    solution[0][i], solution[1][j] = solution[1][j], solution[0][i]

    return solution

def dabc_vrppd(pop_size, nb_bee, max_iter_since_improvement):
    # Initialize the population with random solutions
    solutions = [generate_random_solution() for i in range(pop_size)]

    # Keep track of the best solution and its fitness
    best_total_distance, best_fitness, best_solution = calculate_fitness_vrppd(solutions[0])

    # Initialize the timer
    start_time = time.time()
    iteration_times = []

    # Initialize the number of iterations since the last improvement
    nb_iterations_since_improvement = 0

    while True:
        current_iteration_time = time.time()

        # Research phase (employed bees)
        for i in range(nb_bee):
            new_solution = swap(solutions[i])
            total_distance, fitness, returned_sol = calculate_fitness_vrppd(new_solution)

            # Update the best solution if necessary
            if fitness > best_fitness:
                best_fitness = fitness
                best_solution = returned_sol
                best_total_distance = total_distance
                nb_iterations_since_improvement = 0

            else:
                nb_iterations_since_improvement += 1

        # Research phase (onlooker bees)
        if nb_iterations_since_improvement > max_iter_since_improvement:
            idx = rand.randint(0, nb_bee - 1)
            solutions[idx] = generate_random_solution()
            total_distance, fitness, returned_sol = calculate_fitness_vrppd(solutions[idx])

            # Update the best solution if necessary
            if fitness > best_fitness:
                best_fitness = fitness
                best_solution = returned_sol
                best_total_distance = total_distance
            
            # Reset the number of iterations since the last improvement
            nb_iterations_since_improvement = 0

        # Research phase (scout bees)
        indexes = i, j = rand.sample(range(nb_bee, pop_size), 2)
        for k in range(len(indexes)):
            new_solution = swap(solutions[indexes[k]])
            total_distance, fitness, returned_sol = calculate_fitness_vrppd(new_solution)

            # Update the best solution if necessary
            if fitness > best_fitness:
                best_fitness = fitness
                best_solution = returned_sol
                best_total_distance = total_distance

            else:
                nb_iterations_since_improvement += 1

        # Add the iteration time to the list
        iteration_times.append(time.time() - current_iteration_time)

        if time.time() - start_time > max_time:
            break
    
    return best_solution, best_fitness, best_total_distance, iteration_times

# Résultat
best_solution, best_fitness, best_total_distance, iteration_times = dabc_vrppd(pop_size=50, nb_bee=5, max_iter_since_improvement=10)
print_final_result(best_solution, best_fitness, best_total_distance, iteration_times)

Solution: [[0, 3, 0, 4, 0, 7, 0, 1, 0], [0, 5, 0, 6, 0, 8, 0, 2, 0]]
Fitness: 0.986014

Distance totale : 114.4
Distance drone 1 : 57.4
Distance drone 2 : 57.0

Temps d'exécution : 15 secondes
Nombre d'itérations : 79718

Temps moyen par itération : 0.000188 secondes
Itération par seconde : 5322 itérations par seconde
Itération la plus rapide : 5.3e-05 secondes
Itération la plus longue : 0.011086 secondes


In [48]:
# Discrete Particle Swarm Optimization

# DPSO algorithm for solving VRPPD problems, solution are represented as a list of lists
def dpso_vrppd(swarm_size, w, c1, c2):
    # Initialize the swarm with random solutions
    swarm = [generate_random_solution() for i in range(swarm_size)]

    # Initialize the best solution
    best_total_distance, best_fitness, best_solution = calculate_fitness_vrppd(swarm[0])

    # Initialize particles with velocities
    particles = []
    for i in range(swarm_size):
        particles.append([swarm[i], [rand.randint(-1,1), rand.randint(-1,1)], [0,0]])     # [solution, velocity, position]

    # Initialize the timer
    start_time = time.time()
    iteration_times = []

    # Iterate until the time limit has been reached
    while True:
        current_iteration_time = time.time()
        
        # Evaluate the fitness of each solution
        fitness_scores = []
        for i in range(len(swarm)):
            total_distance, fitness, returned_sol = calculate_fitness_vrppd(swarm[i])
            fitness_scores.append((i,fitness))
            
            # Update the best solution if necessary
            if fitness > best_fitness:
                best_fitness = fitness
                best_solution = returned_sol
                best_total_distance = total_distance

        # Update the velocity of each particle
        for i in range(len(particles)):
            # Choose a random elite solution
            elite_solution = particles[sorted(fitness_scores, key=lambda x: x[1], reverse=True)[0][0]]

            # Generate r1 and r2
            r1 = rand.random()
            r2 = rand.random()

            # Update the velocity of the particle
            for j in range(2):
                cognitive_velocity = c1 * r1 * (elite_solution[1][j] - particles[i][1][j])
                social_velocity = c2 * r2 * (elite_solution[1][j] - particles[i][1][j])
                particles[i][1][j] = (w * particles[i][1][j]) + cognitive_velocity + social_velocity

        # Update the position of each particle
        for i in range(len(particles)):
            for j in range(2):
                particles[i][2][j] += particles[i][1][j]

        # Add the iteration time to the list
        iteration_times.append(time.time() - current_iteration_time)

        # Check if the time limit has been reached
        if time.time() - start_time > max_time:
            break

    return best_solution, best_fitness, best_total_distance, iteration_times

# Résultat
best_solution, best_fitness, best_total_distance, iteration_times = dpso_vrppd(swarm_size=50, w=0.5, c1=1, c2=1)
print_final_result(best_solution, best_fitness, best_total_distance, iteration_times)

Solution: [[0, 6, 0, 7, 0, 8, 0], [0, 1, 0, 2, 0, 3, 0, 4, 0, 5, 0]]
Fitness: 0.972028

Distance totale : 114.4
Distance drone 1 : 57.6
Distance drone 2 : 56.8

Temps d'exécution : 15 secondes
Nombre d'itérations : 15269

Temps moyen par itération : 0.000982 secondes
Itération par seconde : 1018 itérations par seconde
Itération la plus rapide : 8.3e-05 secondes
Itération la plus longue : 0.016466 secondes
